In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

%cd /content/drive/MyDrive/Colab \Notebooks/NLP6010/Project6010 
!ls

/content/drive/MyDrive/Colab Notebooks/NLP6010/Project6010
 build-kenlm.sh       'MIDI Data set exploration'   Project_PlayingAround.ipynb
 ift6010-h21-team1     Mount-test.ipynb		    Quantization.ipynb
 kenlm		       musicautobot		   'Quantization music21.ipynb'
'Meeting notes.gdoc'   Ngram.ipynb


Pull into the shared folder

In [ ]:
path_to_project = "/content/drive/MyDrive/Colab Notebooks/NLP6010/Project6010/ift6010-h21-team1" # @param string
%cd $path_to_project
!git pull

/content/drive/MyDrive/Colab Notebooks/NLP6010/Project6010/ift6010-h21-team1
Already up to date.


# Quantization
Quantization turns the MIDI file into a text file where each token represents a note or some structural indicator.

## Install requirements

In [ ]:
!apt-get install musescore

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon bind9-host desktop-file-utils fluidr3mono-gm-soundfont
  geoclue-2.0 geoip-database gstreamer1.0-plugins-base iio-sensor-proxy
  libavahi-core7 libavahi-glib1 libbind9-160 libbrotli1 libcap2-bin
  libcdparanoia0 libdaemon0 libdns1100 libgeoclue-2-0 libgeoip1
  libgl1-mesa-glx libgstreamer-plugins-base1.0-0 libgstreamer1.0-0 libhyphen0
  libisc169 libisccc160 libisccfg160 liblwres160 libmbim-glib4 libmbim-proxy
  libmm-glib0 libnl-genl-3-200 libnss-mdns liborc-0.4-0 libpam-cap
  libportaudio2 libpulsedsp libqmi-glib5 libqmi-proxy libqt5help5
  libqt5positioning5 libqt5qml5 libqt5quick5 libqt5sensors5 libqt5sql5-sqlite
  libqt5svg5 libqt5webchannel5 libqt5webkit5 libqt5xmlpatterns5
  libvisual-0.4-0 libwoff1 modemmanager musescore-common pulseaudio-utils
  qml-module-qtgraphicaleffects qml-module-qtquick-controls
  qml-module-

In [ ]:
!whereis musescore
from music21 import environment
us = environment.UserSettings()
us['musescoreDirectPNGPath'] = "/usr/bin/musescore" # to let music21 find musescore

musescore: /usr/bin/musescore /usr/share/man/man1/musescore.1.gz


In [ ]:
!pip install musicautobot

## Music autobot score rendering demo
https://github.com/bearpelican/musicautobot/blob/master/notebooks/data_encoding/Midi2Tensor.ipynb


In [ ]:
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.multitask_transformer import *
from musicautobot.utils import midifile

midi_file = Path('data/JSB Chorales/test/102.mid')
vocab = MusicVocab.create()
item = MusicItem.from_file(midi_file, vocab); item.show()

FileNotFoundError: ignored

In [ ]:
item.play()

## MIDI to text
Generates the textual data set as a subfolder inside each train-test-valid split, e.g. `data/JSB Chorales/train/text/*.txt`

In [ ]:
def midi2text(path: str, vocab) -> str:
  # open file
  mf = midifile.file2mf(Path(path))
  stream = file2stream(mf)

  # array of chords
  chordarr = stream2chordarr(stream)

  # First timestep
  npenc = chordarr2npenc(chordarr)
  idxenc = npenc2idxenc(npenc, vocab)
  stringed = vocab.textify(idxenc)
  
  return stringed

# write string forms to disk
import os

JSB_train_path = "data/JSB Chorales/train"
JSB_valid_path = "data/JSB Chorales/valid"
JSB_test_path = "data/JSB Chorales/test"

for folder in [JSB_test_path, JSB_train_path, JSB_valid_path]:
  # create subfolders for output
  output_folder = os.path.join(folder, "text") 
  os.makedirs(output_folder, exist_ok=True)

  for midi in os.listdir(folder):
    
    # only read files
    midi_path = os.path.join(folder, midi)
    if not os.path.isfile(midi_path):
      continue

    # read
    voc = MusicVocab.create()
    MusicItem.from_file(midi_file, voc)
    textified = midi2text(midi_path, vocab=voc)

    # write
    with open(os.path.join(output_folder, os.path.splitext(midi)[0]+'.txt'), 'w') as text_out:
      text_out.write(textified) 


In [ ]:
midi_files = get_files(midi_path, '.mid', recurse=True); len(midi_files)
processors = [Midi2ItemProcessor()]
data = MusicDataBunch.from_files(midi_files, data_path, processors=processors, bs=2, bptt=12)
data.save(data_save_name)

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [ ]:
midi_file = Path('data/JSB Chorales/train/363.mid'); midi_file
item = MusicItem.from_file(midi_file, data.vocab);
item.show()

NameError: ignored